# Graph Auto-Encoders

## Introduction
This tutorial demonstrates the use of Graph Auto-Encoders (GAEs) and Variational Graph Auto-Encoders (VGAEs) to perform unsupervised learning on a graph. The models will be implemented with PyTorch and PyG (torch geometric) (a <a href='https://github.com/tkipf/gae'>tensorflow implementation</a> has been produced by the author of VGAE).
To demonstrate the suitably of the models' embeddings, we show how unsupervised training using only the reconstruction loss improves performance on a link prediction task that the model is not directly optimising for.

## Setup
Here we load required libraries, define paths to data, and define some helper functions. **Feel free to skip this section.**

In [80]:
import torch
from datetime import datetime
from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.datasets import Planetoid
from tensorboardX import SummaryWriter
from torch_geometric.nn.models.autoencoder import EPS, MAX_LOGVAR
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch_geometric.utils import to_networkx, train_test_split_edges
from torch_geometric.data import Data
from torch_geometric.nn.inits import reset
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from pandas import DataFrame as df
import numpy as np
import networkx as nx
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.utils import (negative_sampling, remove_self_loops, add_self_loops)

In [81]:
# define the path to save datasets
fp_data = "./datasets/"

## Load data

### Citeseer
The citeseer graph is accessible through PyG

In [82]:
# load the dataset - small download on first run
dataset = Planetoid(root="./datasets/citeseer", name="Citeseer")
data = dataset[0]

In [83]:
# explore key features of the graph
print('num nodes =', data.num_nodes)
print('num edges =', data.num_edges)
print('num features =', data.num_node_features)

num nodes = 3327
num edges = 9104
num features = 3703


<strong>Useful PyG Citeseer graph attributes</strong>
<table width="75%">
    <header>
        <th style="text-align:left">Name</th>
        <th style="text-align:left">Attribute</th>
        <th style="text-align:left">Shape</th>
    </header>
    <tr>
        <td style="text-align:left">Node Feature Matrix</td>
        <td style="text-align:left">data.x</td>
        <td style="text-align:left">(num nodes, num features)</td>
    </tr>
    <tr>
        <td style="text-align:left">Edge Index</td>
        <td style="text-align:left">data.edge_index</td>
        <td style="text-align:left">(2, num edges)</td>
    </tr>
    <tr>
        <td style="text-align:left">Node Classes</td>
        <td style="text-align:left">data.y</td>
        <td style="text-align:left">(num nodes, )</td>
    </tr>
</table>

## Visualisation

In [84]:
# define a plot function using networkx
def plot(x, y, edge_index):
    """
    plot a graph using networkx
    :param x: feature matrix, shape (n nodes, n features)
    :param y: node classes, shape (n nodes)
    :param edge_index: edge index in coo format, shape (2, n edges)
    """
    graph = to_networkx(Data(x=x, y=y, edge_index=edge_index))
    nx.draw_networkx(graph)

In [85]:
# we can plot this graph, though with 3327 nodes it is not very helpful
#plot(data.x, data.y, data.edge_index)

In [86]:
# instead we can select and plot a smaller sub-graph
#plot(None, None, data.edge_index[:, :10])

## Prepare data for edge prediction

In [87]:
# To perform edge prediction, we need to split the graph's edges intro train and test sets
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)
# Note that data.edge_index is no longer available, as edges have been distributed between train and test
print('num train edges', data.train_pos_edge_index.shape[1])

num train edges 7740


In [88]:
# Get the device to use for training our model
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('CUDA available:', torch.cuda.is_available())

CUDA available: True


In [89]:
# We then register the relevant data to the device in use
x = data.x.to(dev)  # feature matrix
train_pos_edge_index = data.train_pos_edge_index.to(dev)  # index of true positive edges in the train set


## GAE Model
To understand GAEs, we show the models as constructed by PyG. Alternatively, these can be imported directly from torch_geometric.nn
 which contains methods to:
 
Importantly, the GAE class includes methods to:
<ul>
    <li>apply the encoder and decoder</li>
    <li>calculate reconstruction loss</li>
</ul>

In [90]:
# define the GAE class

class GAE(torch.nn.Module):
    r"""The Graph Auto-Encoder model from the
    `"Variational Graph Auto-Encoders" <https://arxiv.org/abs/1611.07308>`_
    paper based on user-defined encoder and decoder models.

    Args:
        encoder (Module): The encoder module.
        decoder (Module, optional): The decoder module. If set to :obj:`None`,
            will default to the
            :class:`torch_geometric.nn.models.InnerProductDecoder`.
            (default: :obj:`None`)
    """
    def __init__(self, encoder, decoder=None):
        super(GAE, self).__init__()
        self.encoder = encoder
        self.decoder = InnerProductDecoder() if decoder is None else decoder
        GAE.reset_parameters(self)

    def reset_parameters(self):
        reset(self.encoder)
        reset(self.decoder)


    def encode(self, *args, **kwargs):
        r"""Runs the encoder and computes node-wise latent variables."""
        return self.encoder(*args, **kwargs)


    def decode(self, *args, **kwargs):
        r"""Runs the decoder and computes edge probabilities."""
        return self.decoder(*args, **kwargs)


    def recon_loss(self, z, pos_edge_index):
        r"""Given latent variables :obj:`z`, computes the binary cross
        entropy loss for positive edges :obj:`pos_edge_index` and negative
        sampled edges.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            pos_edge_index (LongTensor): The positive edges to train against.
        """

        pos_loss = -torch.log(
            self.decoder(z, pos_edge_index, sigmoid=True) + EPS).mean()

        # Do not include self-loops in negative samples
        pos_edge_index, _ = remove_self_loops(pos_edge_index)
        pos_edge_index, _ = add_self_loops(pos_edge_index)

        neg_edge_index = negative_sampling(pos_edge_index, z.size(0))
        neg_loss = -torch.log(1 -
                              self.decoder(z, neg_edge_index, sigmoid=True) +
                              EPS).mean()

        return pos_loss + neg_loss


    def test(self, z, pos_edge_index, neg_edge_index):
        r"""Given latent variables :obj:`z`, positive edges
        :obj:`pos_edge_index` and negative edges :obj:`neg_edge_index`,
        computes area under the ROC curve (AUC) and average precision (AP)
        scores.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            pos_edge_index (LongTensor): The positive edges to evaluate
                against.
            neg_edge_index (LongTensor): The negative edges to evaluate
                against.
        """
        pos_y = z.new_ones(pos_edge_index.size(1))
        neg_y = z.new_zeros(neg_edge_index.size(1))
        y = torch.cat([pos_y, neg_y], dim=0)

        pos_pred = self.decoder(z, pos_edge_index, sigmoid=True)
        neg_pred = self.decoder(z, neg_edge_index, sigmoid=True)
        pred = torch.cat([pos_pred, neg_pred], dim=0)

        y, pred = y.detach().cpu().numpy(), pred.detach().cpu().numpy()

        return roc_auc_score(y, pred), average_precision_score(y, pred)


In [91]:
# define the encoder and decoder that GAE will use
# this code is slightly adapted from the PyG example to accept model_name
class GAEEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAEEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

In [92]:
# To decode embeddings and attempt input reconstruction, we can use the standard InnerProductDecoder
class InnerProductDecoder(torch.nn.Module):
    r"""The inner product decoder from the `"Variational Graph Auto-Encoders"
    <https://arxiv.org/abs/1611.07308>`_ paper

    .. math::
        \sigma(\mathbf{Z}\mathbf{Z}^{\top})

    where :math:`\mathbf{Z} \in \mathbb{R}^{N \times d}` denotes the latent
    space produced by the encoder."""
    def forward(self, z, edge_index, sigmoid=True):
        r"""Decodes the latent variables :obj:`z` into edge probabilities for
        the given node-pairs :obj:`edge_index`.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            sigmoid (bool, optional): If set to :obj:`False`, does not apply
                the logistic sigmoid function to the output.
                (default: :obj:`True`)
        """
        value = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)
        return torch.sigmoid(value) if sigmoid else value

    def forward_all(self, z, sigmoid=True):
        r"""Decodes the latent variables :obj:`z` into a probabilistic dense
        adjacency matrix.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            sigmoid (bool, optional): If set to :obj:`False`, does not apply
                the logistic sigmoid function to the output.
                (default: :obj:`True`)
        """
        adj = torch.matmul(z, z.t())
        return torch.sigmoid(adj) if sigmoid else adj

In [93]:
# bring the encoder and decoder together to compile the GAE
# in_channels (the dimensionality of the input data) is simply the number of features
# we define out_channels (the dimensionality of the latent vector or embedding) as 16
model = GAE(GAEEncoder(in_channels=dataset.num_features, out_channels=16)).to(dev)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [94]:
# define the optimizer with a learning rate of 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [95]:
# define a writer to log training results
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

Although we will train the GAE in an unsupervised manner (using reconstruction loss) <i>we can validate it's performance by testing it's ability to perform link prediction during training </i> (even though this is not used in back propagation).

In [96]:
# define a function to test our predictions against the ground truth edges
def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [97]:
# train the model
for epoch in range(0, 1000):
    optimizer.zero_grad()
    # get the embedding (z) of each node, shape(num_nodes, channels)
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()

    writer.add_scalar("loss", loss.item(), epoch)

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    writer.add_scalar("AUC", auc, epoch)
    writer.add_scalar("AP", ap, epoch)
    if epoch % 100 == 0:
        print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 000, AUC: 0.9666, AP: 0.9680
Epoch: 100, AUC: 0.9596, AP: 0.9576
Epoch: 200, AUC: 0.9510, AP: 0.9476
Epoch: 300, AUC: 0.9488, AP: 0.9459
Epoch: 400, AUC: 0.9483, AP: 0.9465
Epoch: 500, AUC: 0.9456, AP: 0.9443
Epoch: 600, AUC: 0.9463, AP: 0.9423
Epoch: 700, AUC: 0.9445, AP: 0.9431
Epoch: 800, AUC: 0.9452, AP: 0.9400
Epoch: 900, AUC: 0.9512, AP: 0.9465


## VGAE Model
To understand VGAEs, we again show the model as constructed by PyG (torch_geometric.nn.VGAE).
 
VGAE differs from GAE in the use of two GCNs that encode the mean and variance. This allows it to reconstruct unseen inputs but requires a few updates to the main class.
Namely:
<ul>
    <li>we use a KL loss, which uses the reconstruction loss from GAEs but adds a term to measure the probability distributions</li>
    <li>the encoder uses the reparametrisation trick in order to maintain a differentiable loss function </li>
</ul>

In [101]:
# define the VGAE class, which inherits from GAE 

class VGAE(GAE):
    r"""The Variational Graph Auto-Encoder model from the
    `"Variational Graph Auto-Encoders" <https://arxiv.org/abs/1611.07308>`_
    paper.

    Args:
        encoder (Module): The encoder module to compute :math:`\mu` and
            :math:`\log\sigma^2`.
        decoder (Module, optional): The decoder module. If set to :obj:`None`,
            will default to the
            :class:`torch_geometric.nn.models.InnerProductDecoder`.
            (default: :obj:`None`)
    """
    def __init__(self, encoder, decoder=None):
        super(VGAE, self).__init__(encoder, decoder)

    def reparametrize(self, mu, logvar):
        if self.training:
            return mu + torch.randn_like(logvar) * torch.exp(logvar)
        else:
            return mu

    def encode(self, *args, **kwargs):
        """"""
        self.__mu__, self.__logvar__ = self.encoder(*args, **kwargs)
        self.__logvar__ = self.__logvar__.clamp(max=MAX_LOGVAR)
        z = self.reparametrize(self.__mu__, self.__logvar__)
        return z

    def kl_loss(self, mu=None, logvar=None):
        r"""Computes the KL loss, either for the passed arguments :obj:`mu`
        and :obj:`logvar`, or based on latent variables from last encoding.

        Args:
            mu (Tensor, optional): The latent space for :math:`\mu`. If set to
                :obj:`None`, uses the last computation of :math:`mu`.
                (default: :obj:`None`)
            logvar (Tensor, optional): The latent space for
                :math:`\log\sigma^2`.  If set to :obj:`None`, uses the last
                computation of :math:`\log\sigma^2`.(default: :obj:`None`)
        """
        mu = self.__mu__ if mu is None else mu
        logvar = self.__logvar__ if logvar is None else logvar.clamp(
            max=MAX_LOGVAR)
        return -0.5 * torch.mean(
            torch.sum(1 + logvar - mu**2 - logvar.exp(), dim=1))

In [106]:
# define a new encoder which incorporates new GCN layers for embedding the probability distribution
class VGAEEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VGAEEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logvar = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv_mu(x, edge_index), self.conv_logvar(x, edge_index)

In [107]:
# bring the encoder and decoder together to compile the VGAE
# in_channels (the dimensionality of the input data) is simply the number of features
# we define out_channels (the dimensionality of the latent vector or embedding) as 16
model = VGAE(VGAEEncoder(in_channels=dataset.num_features, out_channels=16)).to(dev)

In [108]:
# redefine the optimizer in order to reset it from the GAE training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [109]:
# train the model
for epoch in range(0, 1000):
    optimizer.zero_grad()
    # get the embedding (z) of each node, shape(num_nodes, channels)
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()

    writer.add_scalar("loss", loss.item(), epoch)

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    writer.add_scalar("AUC", auc, epoch)
    writer.add_scalar("AP", ap, epoch)
    if epoch % 100 == 0:
        print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 000, AUC: 0.6816, AP: 0.7143
Epoch: 100, AUC: 0.8644, AP: 0.8791
Epoch: 200, AUC: 0.8548, AP: 0.8708
Epoch: 300, AUC: 0.8414, AP: 0.8578
Epoch: 400, AUC: 0.8409, AP: 0.8598
Epoch: 500, AUC: 0.8430, AP: 0.8583
Epoch: 600, AUC: 0.8552, AP: 0.8681
Epoch: 700, AUC: 0.8603, AP: 0.8688
Epoch: 800, AUC: 0.8584, AP: 0.8650
Epoch: 900, AUC: 0.8502, AP: 0.8597
